In [ ]:
import os
import numpy as np
import torch
import torch.utils.data
import torchvision.transforms as transforms
import PIL
import random
from scipy import ndimage
import glob
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import torch
import torch.nn.functional as F
from torch import nn
from torch.utils.data import DataLoader,Dataset
from torchvision.datasets import CIFAR100,ImageNet
from torchvision import transforms
from time import time
from skimage.data import astronaut,rocket
from skimage.metrics import structural_similarity
from PIL import Image


In [ ]:
addresses_t = glob.glob('../input/imagenetmini-1000/imagenet-mini/train/*/*.JPEG')
addresses_v = glob.glob('../input/imagenetmini-1000/imagenet-mini/val/*/ILSVRC2012_val_*.JPEG')       
train_addressess=[]
val_addressess=[]
for x in addresses_t:
    img = Image.open(x)
    img_shape = transforms.ToTensor()(img).size()
    if ((img_shape[0] == 1) or (img_shape[1] < 256) or (img_shape[2] < 256)):
        pass
    else:
        train_addressess.append(x)
        
        
with open('address_train.txt', 'w') as f:
    for line in train_addressess:
        f.write(line)
        f.write('\n')
f.close()        
                       
for x in addresses_v:
    img = Image.open(x)
    img_shape = transforms.ToTensor()(img).size()
    if ((img_shape[0] == 1) or (img_shape[1] < 256) or (img_shape[2] < 256)):
        pass
    else:
        val_addressess.append(x)
        
with open('address_test.txt', 'w') as f:
    for line in val_addressess:
        f.write(line)
        f.write('\n')
f.close()                

In [ ]:
with open('../input/image-net-steganography/address_train_carrier.txt','r') as f:
    train_addresses_carrier = f.readlines() 
f.close()        
with open('../input/image-net-steganography/address_train_secret.txt','r') as f:
    train_addresses_secret = f.readlines() 
f.close()  
with open('../input/image-net-steganography/address_test_carrier.txt','r') as f:
    test_addresses_carrier = f.readlines() 
f.close()   
with open('../input/image-net-steganography/address_test_secret.txt','r') as f:
    test_addresses_secret = f.readlines() 
f.close()   

In [ ]:
test_addresses_carrier[0]

In [ ]:
# class train_Dataset(torch.utils.data.Dataset):
#     def __init__(self):
#         super(train_Dataset, self).__init__()
# #         self.paths_train=glob.glob('../input/imagenet/imagenet/train/ILSVRC2012_val_*.JPEG')
#         self.paths_train=train_addresses
#         self.paths_train.sort()
#         self.data_files =self.paths_train
        
#     def __getitem__(self,idx):
#         x =Image.open(self.data_files[idx].rstrip())
#         x_tensor = transforms.ToTensor()(x)
#         x_resized=x_tensor[:,0:256,0:256]
#         return x_resized 
# #         x =np.array(Image.open(self.data_files[idx]))
# #         x_resized=x[0:255,0:255]
# #         x_tensor = torch.from_numpy(x_resized)
# #         x_tensor = x_tensor.permute(2,0,1)
#     #         x_tensor = F.normalize(x_tensor)
          
#     def __len__(self):
#         return len(self.data_files)
    
# class test_Dataset(torch.utils.data.Dataset):
#     def __init__(self):
#         super(test_Dataset, self).__init__()
# #         self.paths_test=glob.glob('../input/imagenet/imagenet/val/ILSVRC2012_val_*.JPEG')
#         self.paths_test=test_addresses
#         self.paths_test.sort()
# #         for x in self.paths_test :
# #             img = Image.open(x)
# #             img_shape = transforms.ToTensor()(img).size()
# #             if img_shape[0] == 1:
# #                 pass
# #             else:
#         self.data_files = self.paths_test
#     def __getitem__(self,idx):
#         x =Image.open(self.data_files[idx].rstrip())
#         x_tensor = transforms.ToTensor()(x)
#         x_resized=x_tensor[:,0:256,0:256]
#         return x_resized
    
# #         x_tensor = torch.from_numpy(x_resized)
# #         x_tensor = x_tensor.permute(2,0,1)
#     #         x_tensor = F.normalize(x_tensor) 
#     def __len__(self):
#         return len(self.data_files)



class train_Dataset(torch.utils.data.Dataset):
    def __init__(self):
        super(train_Dataset, self).__init__()
        self.paths_train_carrier=train_addresses_carrier
        self.paths_train_carrier.sort()
        self.data_files_carrier =self.paths_train_carrier
        self.paths_train_secret=train_addresses_secret
        self.paths_train_secret.sort()
        self.data_files_secret =self.paths_train_secret
        
    def __getitem__(self,idx):
        x =Image.open(self.data_files_carrier[idx].rstrip())
        x_tensor = transforms.ToTensor()(x)
        carrier=x_tensor[:,0:256,0:256]
        y =Image.open(self.data_files_secret[idx].rstrip())
        y_tensor = transforms.ToTensor()(y)
        secret=y_tensor[:,0:256,0:256]
        return carrier , secret 
          
    def __len__(self):
        return len(self.data_files_secret)
    
class test_Dataset(torch.utils.data.Dataset):
    def __init__(self):
        super(test_Dataset, self).__init__()
        self.paths_test_carrier=test_addresses_carrier
        self.paths_test_carrier.sort()
        self.data_files_carrier =self.paths_test_carrier
        self.paths_test_secret=test_addresses_secret
        self.paths_test_secret.sort()
        self.data_files_secret =self.paths_test_secret
        
    def __getitem__(self,idx):
        x =Image.open(self.data_files_carrier[idx].rstrip())
        x_tensor = transforms.ToTensor()(x)
        carrier=x_tensor[:,0:256,0:256]
        y =Image.open(self.data_files_secret[idx].rstrip())
        y_tensor = transforms.ToTensor()(y)
        secret=y_tensor[:,0:256,0:256]
        return carrier , secret 
          
    def __len__(self):
        return len(self.data_files_secret)

In [ ]:
# class train_Dataset(torch.utils.data.Dataset):
#     def __init__(self):
#         super(train_Dataset, self).__init__()
#         self.paths_train=train_addresses
#         self.paths_train.sort()
#         self.data_files =self.paths_train
        
#     def __getitem__(self,idx):
#         x =Image.open(self.data_files[idx].rstrip())
#         x_tensor = transforms.ToTensor()(x)
#         x_resized=x_tensor[:,0:256,0:256]
#         return x_resized 
          
#     def __len__(self):
#         return len(self.data_files)
    
# class test_Dataset(torch.utils.data.Dataset):
#     def __init__(self):
#         super(test_Dataset, self).__init__()
#         self.paths_test=test_addresses
#         self.paths_test.sort()
#         self.data_files = self.paths_test
#     def __getitem__(self,idx):
#         x =Image.open(self.data_files[idx].rstrip())
# #         y =Image.open(self.data_files[idx].rstrip()) 
#         x_tensor = transforms.ToTensor()(x)
#         x_resized=x_tensor[:,0:256,0:256]
#         return x_resized
#     def __len__(self):
#         return len(self.data_files)

In [ ]:
train_data=train_Dataset()
test_data=test_Dataset()
train_loader = DataLoader(train_data, batch_size = 8, num_workers = 0)
test_loader = DataLoader(test_data, batch_size = 16, num_workers = 0)

In [ ]:
print(len(test_data))

In [ ]:
for batch_i, (carrier,secret) in enumerate(train_loader):
    fig, ax = plt.subplots(1,2, figsize =(15,15))
    ax[0].imshow(carrier[5].permute(1,2,0))
    ax[1].imshow(secret[2].permute(1,2,0))
    break

In [ ]:
print(len(train_data))
print(len(test_data))

In [ ]:
for batch_i, (carrier,secret) in enumerate(train_loader):
    concatenated_input=torch.cat((carrier,secret),1)
    print(carrier.shape)  
    print(secret.shape)        
    print(concatenated_input.shape)    
    break

In [ ]:
############ Defining classes which are used in Network #############

############ DenseNet is composed of Single layers  #############

class SingleLayer(nn.Module):
    def __init__(self, nChannels, growthRate):
        super(SingleLayer, self).__init__()
        self.bn     = nn.BatchNorm2d(nChannels)
        self.conv   = nn.Conv2d(nChannels, growthRate, kernel_size=3,padding=1, bias=False)
        self.relu   = nn.ReLU()
        self.dropout = nn.Dropout2d(0.2)

    def forward(self, x):
        out = self.bn(x)
        out = self.conv(out)
        out = self.relu(out)
        out = self.dropout(out)
        out = torch.cat((x, out), 1)
        return out
####################################################################################################

############ DenseNet is made by choosing number of layers and growth rate which is a constant output of each layer  #############

class DenseNet(nn.Module):
  def __init__(self, nChannels, growthRate, nDenseBlocks):
    super(DenseNet,self).__init__()
    self.dense = self.make_dense(nChannels, growthRate, nDenseBlocks)
  
  def make_dense(self, nChannels, growthRate, nDenseBlocks):
    layers = []
    for i in range(int(nDenseBlocks)):
        layers.append(SingleLayer(nChannels, growthRate))
        nChannels += growthRate
    return nn.Sequential(*layers)  

  def forward(self,x):
    out = self.dense(x)
    return out


####################################################################################################

############ Transition down layer is used to reduce the number of feature maps  #############

class Transition_Down(nn.Module): 
  def __init__(self, input_channels,output_channels):
    super(Transition_Down, self).__init__()
    self.TD = nn.Sequential(nn.BatchNorm2d(input_channels),
                            nn.ReLU(),
                            nn.Conv2d(input_channels,output_channels,kernel_size=1,padding ='same'),
                            nn.MaxPool2d(kernel_size=2,stride=2))
    
  def forward(self,x):
    out = self.TD(x)
    return out  
    
####################################################################################################

############ Transition up layer is used to compensate the effect of transtion down layer which has reduce nemuber of feature maps  #############

class Transition_Up(nn.Module): 
  def __init__(self, input_channels,output_channels):
    super(Transition_Up, self).__init__()
    self.TU = nn.ConvTranspose2d(input_channels,output_channels, kernel_size=2, stride=2 , padding=0)
    
  def forward(self,x):
    out = self.TU(x)
    return out  
    
####################################################################################################

############ Decoder block is used in reveal network ro extract secret image #############
class Decoder_block(nn.Module): 
  def __init__(self, input_channels , output_channels):
    super(Decoder_block, self).__init__()
    self.DecoderBlock = nn.Sequential(nn.Conv2d(input_channels , output_channels , kernel_size=3 , padding =1 ,stride=1),
                            nn.BatchNorm2d(output_channels),
                            nn.ReLU())
    
  def forward(self,x):
    out = self.DecoderBlock(x)
    return out      
    

In [ ]:
############ Defining Hidden Network by using the classes which are defined previously #############

class Hidden_Network(nn.Module):
  def __init__(self):
    super(Hidden_Network,self).__init__()
    self.conv1 = nn.Conv2d( 6, 48, kernel_size = 3 , padding = 'same')
    self.DB1   = DenseNet (48,12,4)
    self.TD1   = Transition_Down(144,96)
    self.DB2   = DenseNet (96,12,4)
    self.TD2   = Transition_Down(240,144)
    self.DB3   = DenseNet (144,12,4)
    self.TD3   = Transition_Down(336,192)
    self.DB4   = DenseNet (192,12,4)
    self.TD4   = Transition_Down(432,240)
    self.DB5   = DenseNet (240,12,4)
    self.TD5   = Transition_Down(528,288)
    self.DB6   = DenseNet (288,12,4)
    self.TU1   = Transition_Up (336,336)
    self.DB7   = DenseNet (864,12,4)
    self.TU2   = Transition_Up (912,112)
    self.DB8   = DenseNet (544,12,4)
    self.TU3   = Transition_Up (592,92)
    self.DB9   = DenseNet (428,12,4)
    self.TU4   = Transition_Up (476,76)
    self.DB10  = DenseNet (316,12,4)
    self.TU5   = Transition_Up (364,64)
    self.DB11  = DenseNet (208,12,4)
    self.conv2 = nn.Conv2d( 256, 3, kernel_size = 3 , padding = 'same')
  def forward(self, input):
    y   = self.conv1(input)
    DB1 = self.DB1(y)
    C1  = torch.cat((y, DB1), 1)
    TD1 = self.TD1(C1)
    DB2 = self.DB2(TD1)
    C2  = torch.cat((DB2, TD1), 1)
    TD2 = self.TD2(C2)
    DB3 = self.DB3(TD2)
    C3  = torch.cat((DB3, TD2), 1)
    TD3 = self.TD3(C3)
    DB4 = self.DB4(TD3)
    C4  = torch.cat((DB4, TD3), 1)
    TD4 = self.TD4(C4)
    DB5 = self.DB5(TD4)
    C5  = torch.cat((DB5, TD4), 1)
    TD5 = self.TD5(C5)
    DB6 = self.DB6(TD5)
    TU1 = self.TU1(DB6)
    C6  = torch.cat((TU1, C5), 1)
    DB7 = self.DB7(C6)
    TU2 = self.TU2(DB7)
    C7  = torch.cat((TU2, C4), 1)
    DB8 = self.DB8(C7)
    TU3 = self.TU3(DB8)
    C8  = torch.cat((TU3, C3), 1)
    DB9 = self.DB9(C8)
    TU4 = self.TU4(DB9)
    C9  = torch.cat((TU4, C2), 1)
    DB10= self.DB10(C9)
    TU5 = self.TU5(DB10)
    C10 = torch.cat((TU5, C1), 1)
    DB11= self.DB11(C10)
    output = self.conv2(DB11)
    return output

In [ ]:
########## Testing the Hidden Network ###########
a = torch.rand(1,6,256,256)
Hidden_Network().forward(a).shape



In [ ]:
############ Defining Reveal Network by using the classes which are defined previously #############

class Reveal_Network(nn.Module):

  def __init__(self):
    super(Reveal_Network,self).__init__()
    self.Decoder1 = Decoder_block(3,64)
    self.Decoder2 = Decoder_block(64,128)
    self.Decoder3 = Decoder_block(128,256)
    self.Decoder4 = Decoder_block(256,128)
    self.Decoder5 = Decoder_block(128,64)
    self.conv1    = nn.Conv2d( 64 , 3 , kernel_size=3 , padding =1 ,stride=1)
    self.sigmo    = nn.Sigmoid()

  def forward(self, input):
    Decoder1  = self.Decoder1(input)
    Decoder2  = self.Decoder2(Decoder1)
    Decoder3  = self.Decoder3(Decoder2)
    Decoder4  = self.Decoder4(Decoder3)
    Decoder5  = self.Decoder5(Decoder4)
    output    = self.sigmo(self.conv1(Decoder5))
    
    return output

In [ ]:
########## Testing the Reveal Network ###########
a = torch.rand(1,3,256,256)
Reveal_Network().forward(a).shape

In [ ]:
class Stegano_Network(nn.Module):

  def __init__(self):
    super(Stegano_Network,self).__init__()
    self.hidden=Hidden_Network()
    self.reveal=Reveal_Network()
  def forward(self, input):
    Stego = self.hidden(input)
    revealed_image = self.reveal(Stego)
    
    return Stego,revealed_image

In [ ]:
class EncoderLoss(nn.Module):
  def __init__(self):
      super(EncoderLoss,self).__init__()
        
  def forward(self,carrier_image,stego_image):    
      encoder_loss = (F.mse_loss(carrier_image,stego_image))
      return encoder_loss

class DecoderLoss(nn.Module):
  def __init__(self):
      super(DecoderLoss,self).__init__()
        
  def forward(self,revealed_image,secret_image):
      decoder_loss = (F.mse_loss(revealed_image,secret_image))
      return decoder_loss
class SteganoLoss(nn.Module):
  def __init__(self,decoder_weight):
      super(SteganoLoss,self).__init__()
      self.decoder_weight = decoder_weight
        
  def forward(self,carrier_image,stego_image,revealed_image,secret_image):
      stegano_loss = (F.mse_loss(carrier_image,stego_image)) + (self.decoder_weight*(F.mse_loss(revealed_image,secret_image)))
      return stegano_loss     


In [ ]:
device = ('cuda' if torch.cuda.is_available() else 'cpu')
mymodel= Stegano_Network().to(device)

In [ ]:
epochs = 10
criterion_encoder=EncoderLoss()
criterion_decoder=DecoderLoss()
criterion = SteganoLoss(0.85)   # 0.75 is beta hyperparamete in the paper
opt = torch.optim.Adam(mymodel.parameters())

encoder_losses = [] # hidden Losses in different epochs
decoder_losses = [] # reveal Losses in different epochs
losses        = [] # whole network Losses in different epochs
# carrier=[]
# secret=[]
# training_part = 0 # training_part = 0 : hidden network is being trianed //// training_part = 1 : reveal network is being trianed
for epoch in range(epochs):    
    for batch_i, (carrier,secret) in enumerate(train_loader):
        tic = time() # Start of epoch
        # Train
        mymodel = mymodel.train()
        batch_losses = []
        encoder_batch_losses = []
        decoder_batch_losses = []
        carrier=carrier.to(device)
        secret = secret.to(device)
        concatenated_input=torch.cat((carrier,secret),1)
        concatenated_input = concatenated_input.to(device)
        Stego_image,revealed_image = mymodel.forward(concatenated_input)
        encoder_loss=criterion_encoder(carrier,Stego_image)
        decoder_loss=criterion_decoder(revealed_image,secret)
        opt.zero_grad()
        loss = criterion.forward(Stego_image, carrier,revealed_image,secret)
        loss.backward()
        opt.step()
        batch_losses.append(loss.item())
        encoder_batch_losses.append(encoder_loss.item())
        decoder_batch_losses.append(decoder_loss.item())

    epoch_loss = np.mean(batch_losses)
    encoder_epoch_loss = np.mean(encoder_batch_losses)
    decoder_epoch_loss = np.mean(decoder_batch_losses)
    losses.append(epoch_loss)
    train_string = f"Epoch : {epoch} // loss : {epoch_loss: 0.7f} // encoder_loss : {encoder_epoch_loss: 0.7f} // decoder_loss : {decoder_epoch_loss: 0.7f} "
    print(train_string )


      # mymodel = mymodel.eval() ### To disable dropout
      # # Validation
      # batch_losses = []
      # for batch_i, (inputs, _) in enumerate(test_loader):
      #   inputs = inputs.to(device) # Images
      #   size=inputs.size()
      #   carrier=torch.empty(int(size[0]/2),size[1],size[2],size[3])
      #   secret=torch.empty(int(size[0]/2),size[1],size[2],size[3])
      #   concatenated_input=torch.empty(int(size[0]/2),2*size[1],size[2],size[3])
      #   j=0
      #   z=0
      #   for i in range(len(inputs)):
      #     if i%2 == 0 : 
      #       carrier[j]=inputs[i]
      #       j=j+1
      #     else :
      #       secret[z]=inputs[i]
      #       concatenated_input[z] = torch.cat((carrier[j-1],secret[z]),0)
      #       z=z+1

      #   concatenated_input = concatenated_input.to(device)
      #   carrier=carrier.to(device)
      #   secret=secret.to(device)
      #   with torch.no_grad():
      #     output = mymodel.forward(concatenated_input)
      #     loss = criterion(output, carrier)
      #     batch_losses.append(loss.item())

      # epoch_loss = np.mean(batch_losses)
      # val_losses.append(epoch_loss)
      # toc = time() #End of epoch
      # val_string = f" ////////////// val_loss : {epoch_loss: 0.5f}, , time : {toc - tic: 0.1f}"
      
    


In [ ]:
# epochs = 10
# criterion_encoder=EncoderLoss()
# criterion_decoder=DecoderLoss()
# criterion = SteganoLoss(0.85)   # 0.75 is beta hyperparamete in the paper
# opt = torch.optim.Adam(mymodel.parameters())

# encoder_losses = [] # hidden Losses in different epochs
# decoder_losses = [] # reveal Losses in different epochs
# losses        = [] # whole network Losses in different epochs
# # carrier=[]
# # secret=[]
# # training_part = 0 # training_part = 0 : hidden network is being trianed //// training_part = 1 : reveal network is being trianed
# for epoch in range(epochs):
#       for inputs in (train_loader):
#         tic = time() # Start of epoch
#         # Train
#         mymodel = mymodel.train()
#         batch_losses = []
#         encoder_batch_losses = []
#         decoder_batch_losses = []
#         inputs = inputs.to(device) # Images
#         size=inputs.size()
#         carrier=torch.empty(int(size[0]/2),size[1],size[2],size[3])
#         secret=torch.empty(int(size[0]/2),size[1],size[2],size[3])
#         concatenated_input=torch.empty(int(size[0]/2),2*size[1],size[2],size[3])
#         j=0
#         z=0
#         for i in range(len(inputs)):
#               if i%2 == 0 : 
#                 carrier[j]=inputs[i]
#                 j=j+1
#               else :
#                     secret[z]=inputs[i]
#                     concatenated_input[z] = torch.cat((carrier[j-1],secret[z]),0)
#                     z=z+1

#         concatenated_input = concatenated_input.to(device)
#         carrier=carrier.to(device)
#         secret=secret.to(device)

#         Stego_image,revealed_image = mymodel.forward(concatenated_input)
#         encoder_loss=criterion_encoder(carrier,Stego_image)
#         decoder_loss=criterion_decoder(revealed_image,secret)
#         opt.zero_grad()
#         loss = criterion.forward(Stego_image, carrier,revealed_image,secret)
#         loss.backward()
#         opt.step()
#         batch_losses.append(loss.item())
#         encoder_batch_losses.append(encoder_loss.item())
#         decoder_batch_losses.append(decoder_loss.item())


#       epoch_loss = np.mean(batch_losses)
#       encoder_epoch_loss = np.mean(encoder_batch_losses)
#       decoder_epoch_loss = np.mean(decoder_batch_losses)
#       losses.append(epoch_loss)
#       train_string = f"Epoch : {epoch} // loss : {epoch_loss: 0.7f} // encoder_loss : {encoder_epoch_loss: 0.7f} // decoder_loss : {decoder_epoch_loss: 0.7f} "


#       # mymodel = mymodel.eval() ### To disable dropout
#       # # Validation
#       # batch_losses = []
#       # for batch_i, (inputs, _) in enumerate(test_loader):
#       #   inputs = inputs.to(device) # Images
#       #   size=inputs.size()
#       #   carrier=torch.empty(int(size[0]/2),size[1],size[2],size[3])
#       #   secret=torch.empty(int(size[0]/2),size[1],size[2],size[3])
#       #   concatenated_input=torch.empty(int(size[0]/2),2*size[1],size[2],size[3])
#       #   j=0
#       #   z=0
#       #   for i in range(len(inputs)):
#       #     if i%2 == 0 : 
#       #       carrier[j]=inputs[i]
#       #       j=j+1
#       #     else :
#       #       secret[z]=inputs[i]
#       #       concatenated_input[z] = torch.cat((carrier[j-1],secret[z]),0)
#       #       z=z+1

#       #   concatenated_input = concatenated_input.to(device)
#       #   carrier=carrier.to(device)
#       #   secret=secret.to(device)
#       #   with torch.no_grad():
#       #     output = mymodel.forward(concatenated_input)
#       #     loss = criterion(output, carrier)
#       #     batch_losses.append(loss.item())

#       # epoch_loss = np.mean(batch_losses)
#       # val_losses.append(epoch_loss)
#       # toc = time() #End of epoch
#       # val_string = f" ////////////// val_loss : {epoch_loss: 0.5f}, , time : {toc - tic: 0.1f}"
#       print(train_string )
      
    


In [ ]:
torch.save(mymodel.state_dict(), '/content/drive/MyDrive/Saved_Stegano_model_onCIFAR100_ORG.pt')

In [ ]:
'''
 here you can use my saved model that I have sent just upload it to your colab and then write its directory
 here -> (torch.load(directory))
'''
mysaved_model=Stegano_Network()
mysaved_model.load_state_dict(torch.load('/content/drive/MyDrive/Saved_Stegano_model_onCIFAR100_ORG.pt',map_location=torch.device('cpu')))

In [ ]:
############# Visual & Quantative evaluation ###############

############# here we use our test images to evaluate our network #############

MSE = DecoderLoss()

for batch_i, (inputs, _) in enumerate(test_loader):
  x=inputs
  break
mymodel=mymodel.to('cpu')
c1 = torch.cat((x[10],x[11]),0)  # test images have been selected randomly
c2 = torch.cat((x[3],x[12]),0)

c_reshape1=torch.reshape(c1,(1,6,256,256))
c_reshape2=torch.reshape(c2,(1,6,256,256))
stego1,revealed1=mymodel.forward(c_reshape1)
stego2,revealed2=mymodel.forward(c_reshape2)
stego_x1=stego1.detach()
stego_x2=stego2.detach()
revealed_x1=revealed1.detach()
revealed_x2=revealed2.detach()
secret1 = torch.reshape(x[11],(1,3,256,256))
secret2 = torch.reshape(x[12],(1,3,256,256))
carrier1 = torch.reshape(x[10],(1,3,256,256))
carrier2 = torch.reshape(x[3],(1,3,256,256))

''' 
Quantative evaluation methods :
    MSE
    PSNR
    SSIM
'''
#### SSIM ###
ssim = StructuralSimilarityIndexMeasure()
ssim_r_s1= ssim(revealed1, secret1)
ssim_r_s2= ssim(revealed2, secret2)
ssim_c_s1= ssim(stego1, carrier1)
ssim_c_s2= ssim(stego2, carrier2)
#### MSE  ###
mse_r_S1 = MSE(revealed1, secret1)
mse_r_S2 = MSE(revealed2, secret2)
mse_c_S1 = MSE(stego1, carrier1)
mse_c_S2 = MSE(stego2, carrier2)
#### PSNR ### 
revealed1_f = tf.image.convert_image_dtype(revealed1.detach().numpy(), tf.float32)
revealed2_f = tf.image.convert_image_dtype(revealed2.detach().numpy(), tf.float32)
secret1_f = tf.image.convert_image_dtype(secret1.detach().numpy(), tf.float32) 
secret2_f = tf.image.convert_image_dtype(secret2.detach().numpy(), tf.float32) 
stego1_f = tf.image.convert_image_dtype(stego1.detach().numpy(), tf.float32)
stego2_f = tf.image.convert_image_dtype(stego2.detach().numpy(), tf.float32)
carrier1_f = tf.image.convert_image_dtype(carrier1.detach().numpy(), tf.float32) 
carrier2_f = tf.image.convert_image_dtype(carrier2.detach().numpy(), tf.float32) 

psnr_r_s1= tf.image.psnr(revealed1_f,secret1_f,max_val=1)
psnr_r_s2= tf.image.psnr(revealed2_f,secret2_f,max_val=1)
psnr_c_s1= tf.image.psnr(stego1_f,carrier1_f,max_val=1)
psnr_c_s2= tf.image.psnr(stego2_f,carrier2_f,max_val=1)

print(f" Carrier vs Stego :: ssim : {ssim_c_s1} , MSE : {mse_c_S1}  , PSNR : {psnr_c_s1} //// Revealed vs Secret :: ssim : {ssim_r_s1} , MSE : {mse_r_S1}  , PSNR : {psnr_r_s1}")
print(f" Carrier vs Stego :: ssim : {ssim_c_s2} , MSE : {mse_c_S2}  , PSNR : {psnr_c_s2} //// Revealed vs Secret :: ssim : {ssim_r_s2} , MSE : {mse_r_S2}  , PSNR : {psnr_r_s2}")

fig, ax = plt.subplots(1,4, figsize =(15,15))
ax[0].imshow(x[10].permute(1,2,0))
ax[1].imshow(x[11].permute(1,2,0))
ax[2].imshow(stego_x1[0].permute(1,2,0))
ax[3].imshow(revealed_x1[0].permute(1,2,0))
ax[0].set_title("Cover Image")
ax[1].set_title("Secret Image")
ax[2].set_title("Stego Image")
ax[3].set_title("revealed Image")
fig, ax = plt.subplots(1,4, figsize =(15,15))
ax[0].imshow(x[3].permute(1,2,0))
ax[1].imshow(x[12].permute(1,2,0))
ax[2].imshow(stego_x2[0].permute(1,2,0))
ax[3].imshow(revealed_x2[0].permute(1,2,0))
ax[0].set_title("Cover Image")
ax[1].set_title("Secret Image")
ax[2].set_title("Stego Image")
ax[3].set_title("revealed Image")

In [ ]:
############# Visual & Quantative evaluation ###############

############# here we use our test images to evaluate our network #############

MSE = DecoderLoss()

for batch_i, (inputs, _) in enumerate(test_loader):
  x=inputs
  break
mymodel=mymodel.to('cpu')
c1 = torch.cat((x[10],x[11]),0)  # test images have been selected randomly
c2 = torch.cat((x[3],x[12]),0)

c_reshape1=torch.reshape(c1,(1,6,256,256))
c_reshape2=torch.reshape(c2,(1,6,256,256))
stego1,revealed1=mymodel.forward(c_reshape1)
stego2,revealed2=mymodel.forward(c_reshape2)
stego_x1=stego1.detach()
stego_x2=stego2.detach()
revealed_x1=revealed1.detach()
revealed_x2=revealed2.detach()
secret1 = torch.reshape(x[11],(1,3,256,256))
secret2 = torch.reshape(x[12],(1,3,256,256))
carrier1 = torch.reshape(x[10],(1,3,256,256))
carrier2 = torch.reshape(x[3],(1,3,256,256))

''' 
Quantative evaluation methods :
    MSE
    PSNR
    SSIM
'''
#### SSIM ###
# ssim = StructuralSimilarityIndexMeasure()
# ssim_r_s1= ssim(revealed1, secret1)
# ssim_r_s2= ssim(revealed2, secret2)
# ssim_c_s1= ssim(stego1, carrier1)
# ssim_c_s2= ssim(stego2, carrier2)
#### MSE  ###
mse_r_S1 = MSE(revealed1, secret1)
mse_r_S2 = MSE(revealed2, secret2)
mse_c_S1 = MSE(stego1, carrier1)
mse_c_S2 = MSE(stego2, carrier2)
#### PSNR ### 
revealed1_f = tf.image.convert_image_dtype(revealed1.detach().numpy(), tf.float32)
revealed2_f = tf.image.convert_image_dtype(revealed2.detach().numpy(), tf.float32)
secret1_f = tf.image.convert_image_dtype(secret1.detach().numpy(), tf.float32) 
secret2_f = tf.image.convert_image_dtype(secret2.detach().numpy(), tf.float32) 
stego1_f = tf.image.convert_image_dtype(stego1.detach().numpy(), tf.float32)
stego2_f = tf.image.convert_image_dtype(stego2.detach().numpy(), tf.float32)
carrier1_f = tf.image.convert_image_dtype(carrier1.detach().numpy(), tf.float32) 
carrier2_f = tf.image.convert_image_dtype(carrier2.detach().numpy(), tf.float32) 

psnr_r_s1= tf.image.psnr(revealed1_f,secret1_f,max_val=1)
psnr_r_s2= tf.image.psnr(revealed2_f,secret2_f,max_val=1)
psnr_c_s1= tf.image.psnr(stego1_f,carrier1_f,max_val=1)
psnr_c_s2= tf.image.psnr(stego2_f,carrier2_f,max_val=1)

print(f" Carrier vs Stego :: MSE : {mse_c_S1}  , PSNR : {psnr_c_s1} //// Revealed vs Secret ::  , MSE : {mse_r_S1}  , PSNR : {psnr_r_s1}")
print(f" Carrier vs Stego :: MSE : {mse_c_S2}  , PSNR : {psnr_c_s2} //// Revealed vs Secret :: , MSE : {mse_r_S2}  , PSNR : {psnr_r_s2}")

fig, ax = plt.subplots(1,4, figsize =(15,15))
ax[0].imshow(x[10].permute(1,2,0))
ax[1].imshow(x[11].permute(1,2,0))
ax[2].imshow(stego_x1[0].permute(1,2,0))
ax[3].imshow(revealed_x1[0].permute(1,2,0))
ax[0].set_title("Cover Image")
ax[1].set_title("Secret Image")
ax[2].set_title("Stego Image")
ax[3].set_title("revealed Image")
fig, ax = plt.subplots(1,4, figsize =(15,15))
ax[0].imshow(x[3].permute(1,2,0))
ax[1].imshow(x[12].permute(1,2,0))
ax[2].imshow(stego_x2[0].permute(1,2,0))
ax[3].imshow(revealed_x2[0].permute(1,2,0))
ax[0].set_title("Cover Image")
ax[1].set_title("Secret Image")
ax[2].set_title("Stego Image")
ax[3].set_title("revealed Image")